# Analysis of the extended data of player for 2023-2024

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [6]:
data = pd.read_csv('../../data/raw/player_season_stats_23-24.csv')

In [7]:
len(data.columns)

178

In [9]:
selected_columns = [
    # --- Metadata (Needed for identification) ---
    'league', 'season', 'team', 'player', 'nation', 'pos', 'age', 'MP Playing Time','Min Playing Time' ,'90s Playing Time',

    # --- Attack Generation (Goals & Assists) ---
    'npxG Per 90 Minutes',      # The gold standard for scoring threat (Non-Penalty xG)
    'G-PK Per 90 Minutes',      # Actual non-penalty goal output
    'xAG Per 90 Minutes',       # Quality of expected assists
    'xG+xAG Per 90 Minutes',    # Total offensive production

    # --- Shooting Style ---
    'Sh/90 Standard',           # Shooting Volume
    'SoT% Standard',            # Accuracy (Shots on Target %)
    'Dist Standard',            # Average Distance (Distinguishes poachers from range shooters)
    'G/Sh Standard',            # Efficiency (Conversion rate)

    # --- Playmaking & Passing ---
    'KP',                       # Key Passes (Volume of chances created)
    'PrgP',                     # Progressive Passes (Verticality)
    'PPA',                      # Passes into the 18-yard box
    'CrsPA',                    # Crosses into the 18-yard box
    '1/3',                      # Passes into the final third
    'Cmp% Total',               # General retention/safety
    'Cmp% Long',                # Long passing quality
    'Att Long',                 # Long passing intent (Style indicator)

    # --- Pass Types (Tactical Role) ---
    'TB Pass Types',            # Through Balls (Playmaker indicator)
    'Sw Pass Types',            # Switches of play (Range/Distribution)
    'Crs Pass Types',           # Crosses (Winger style)

    # --- Dribbling & Ball Carrying ---
    'PrgC Carries',             # Progressive Carries (Driving forward)
    'CPA Carries',              # Carries into the penalty area
    'Succ Take-Ons',            # Completed Dribbles
    'Succ% Take-Ons',           # Dribble efficiency
    'Dis Carries',              # Dispossessed (Tackled by opponent)
    'Mis Carries',              # Miscontrols (Unforced errors/Clumsiness)
    'PrgR Receiving',           # Progressive Passes Received (Outlet/Target capability)

    # --- Defense & Pressing (Zone Definition) ---
    'TklW Tackles',             # Total Tackles Won
    'Att 3rd Tackles',          # High Pressing (Forwards/High midfielders)
    'Mid 3rd Tackles',          # Midfield engagement
    'Def 3rd Tackles',          # Deep defending
    'Int',                      # Interceptions (Game reading)
    'Sh Blocks',                # Shots blocked (Protective defending)
    'Recov Performance',        # Ball Recoveries (Hustle/Work rate)

    # --- Aerial & Physical ---
    'Won Aerial Duels',         # Volume of aerials won
    'Won% Aerial Duels',        # Aerial Dominance
    'Fls Performance',          # Fouls committed (Aggression)
    'Fld Performance',          # Fouls drawn

    # --- Heatmap Proxies (Positioning - Vital for Clustering) ---
    'Att Pen Touches',          # Touches in opponent's box
    'Att 3rd Touches',          # Touches in offensive zone
    'Mid 3rd Touches',          # Touches in midfield
    'Def 3rd Touches',          # Touches in build-up zone
    'Def Pen Touches',          # Touches in own box
    
    # -- Overall Activity ---
    'Live Touches',              # Overall involvement  
    'Cmp Total'                  # Total Passes Completed
]

In [10]:
data_relevant = data[selected_columns].copy()

In [ ]:
data_relevant.columns

In [14]:
data_relevant.to_csv('../../data/interim/player_season_stats_23-24_relevant.csv', index=False)